In [ ]:
import pandas as pd

# included_scc = ['Educational Services', 'Professional and Business Services', 'Finance and Insurance', 'Full-Service Restaurants', 'Limited-Service Restaurants', 'Supermarkets and Other Grocery (except Convenience) Stores']
# cat_to_scc = {'education': ['Educational Services'], 
#               'office': ['Professional and Business Services', 'Finance and Insurance'], 
#               'social': ['Full-Service Restaurants', 'Limited-Service Restaurants'],
#               'supermarket': ['Supermarkets and Other Grocery (except Convenience) Stores']
#              }

# scc_to_cat = {'Educational Services': 'education',
#               'Professional and Business Services': 'office',
#               'Finance and Insurance': 'office',
#               'Full-Service Restaurants': 'social',
#               'Limited-Service Restaurants': 'social',
#               'Supermarkets and Other Grocery (except Convenience) Stores': 'supermarket'
#              }
cats = ['education','office','retail','social','supermarket']
scc_to_cat = {}
for cat in cats:
    filename = '/Users/lauramiron/Desktop/CEE218/simulator/sim/lib/data/beta_mult/'+cat+'.csv'
    lines = []
    with open(filename,'r') as f:
        lines = [line.strip() for line in f.readlines()]
    for line in lines:
        scc_to_cat[line] = cat

scc_to_cat

In [ ]:
tb_weekday = pd.read_csv('../../covid19/SCC_dashboard/weekday_csv.csv')
tb_weekend = pd.read_csv('../../covid19/SCC_dashboard/weekend_csv.csv')
tb_weekday

In [ ]:
import datetime, dateutil

# tb_weekday['weekday_visits'] = tb_weekday['visits'] + tb_weekday['new_visits']
tb_weekday['weekday_visits'] = tb_weekday['visits']
tb_weekday['week_date'] = tb_weekday.apply(lambda x: dateutil.parser.parse(x['date']) - datetime.timedelta(days=2), axis=1)
tb_weekday = tb_weekday[['date','week_date','scc_category','weekday_visits']].loc[tb_weekday['scc_category'].isin(scc_to_cat.keys())]
tb_weekday['model_category'] = tb_weekday['scc_category'].map(scc_to_cat)

# tb_weekend['weekend_visits'] = tb_weekend['visits'] + tb_weekend['new_visits']
tb_weekend['weekend_visits'] = tb_weekend['visits']
tb_weekend['week_date'] = tb_weekend.apply(lambda x: dateutil.parser.parse(x['date']) - datetime.timedelta(days=6), axis=1)
tb_weekend = tb_weekend[['date','week_date','scc_category','weekend_visits']].loc[tb_weekend['scc_category'].isin(scc_to_cat.keys())]
tb_weekend['model_category'] = tb_weekend['scc_category'].map(scc_to_cat)

# tb_week = pd.merge(tb_weekday,tb_weekend,left_on=['date'])
# tb_week

In [ ]:
tb_weekday_grouped = tb_weekday.groupby(['date','week_date','model_category'],as_index=False).sum()
tb_weekend_grouped = tb_weekend.groupby(['date','week_date','model_category'],as_index=False).sum()


tb_weekday_grouped[0:30]

In [ ]:
tb_weekend_grouped

In [ ]:
tb_week = pd.merge(tb_weekday_grouped,tb_weekend_grouped,how='outer',left_on=['week_date','model_category'],right_on=['week_date','model_category'],left_index=False,right_index=False)
tb_week[0:20]

In [ ]:
tb_week['total_visits'] = ((5*tb_week['weekday_visits']) + (2*tb_week['weekend_visits'])) /2
tb_week

In [ ]:
base_rates = {}
base_rates['education'] = tb_week.loc[tb_weekday_grouped['model_category'] == 'education']['total_visits'].max()
base_rates['office'] = tb_week.loc[tb_weekday_grouped['model_category'] == 'office']['total_visits'].max()
base_rates['retail'] = tb_week.loc[tb_weekday_grouped['model_category'] == 'retail']['total_visits'].max()
base_rates['social'] = tb_week.loc[tb_weekday_grouped['model_category'] == 'social']['total_visits'].max()
base_rates['supermarket'] = tb_week.loc[tb_weekday_grouped['model_category'] == 'supermarket']['total_visits'].max()
base_rates



In [12]:
tb_week['base_rate'] = tb_week['model_category'].map(base_rates)
tb_week['multiplier'] = tb_week['total_visits'] / tb_week['base_rate']
tb_week


,date_x,week_date,model_category,weekday_visits,date_y,weekend_visits,total_visits,base_rate,multiplier
0,2020-01-01,2019-12-30,education,2981,2020-01-05,5606,13058.5,75050.5,0.173996
1,2020-01-01,2019-12-30,office,754,2020-01-05,878,2763.0,7583.0,0.364368
2,2020-01-01,2019-12-30,retail,15184,2020-01-05,20597,58557.0,104534.0,0.560172
3,2020-01-01,2019-12-30,social,18676,2020-01-05,24140,70830.0,111885.5,0.633058
4,2020-01-01,2019-12-30,supermarket,3158,2020-01-05,4628,12523.0,17606.0,0.711292
...,...,...,...,...,...,...,...,...,...
265,2021-01-06,2021-01-04,education,3175,2021-01-10,1800,9737.5,75050.5,0.129746
266,2021-01-06,2021-01-04,office,908,2021-01-10,435,2705.0,7583.0,0.356719
267,2021-01-06,2021-01-04,retail,12796,2021-01-10,9946,41936.0,104534.0,0.401171
268,2021-01-06,2021-01-04,social,10849,2021-01-10,9460,36582.5,111885.5,0.326964


In [13]:
tb_week.to_csv('lib/data/beta_mult/beta_mult_sf.csv')

In [ ]:
# tb_weekday_grouped['base_rate'] = tb_weekday_grouped['model_category'].map(base_rates)
# tb_weekday_grouped['multiplier'] = tb_weekday_grouped['total_visits'] / tb_weekday_grouped['base_rate']
# tb_weekday_grouped


In [ ]:
# tb_weekday_grouped.to_csv('lib/data/beta_mult/beta_mult_sf.csv')

In [ ]:
# tb_weekday_pstay = tb_weekday_clean.groupby(['date'],as_index=False).sum()
# base_rate = tb_weekday_pstay['total_visits'].max()
# base_rate

In [ ]:
# import pandas as pd
# tb_weekday_pstay = pd.read_csv('lib/data/beta_mult/pstay_home.csv')
# tb_weekday_pstay['multiplier'].mean()

In [ ]:
# tb_weekday_pstay['multiplier'] = tb_weekday_pstay['total_visits'] / base_rate
# tb_weekday_pstay

In [ ]:
# tb_weekday_pstay.to_csv('lib/data/beta_mult/pstay_home.csv')